<a href="https://colab.research.google.com/github/mLucas55/spam-email-classification/blob/main/spam_email_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [ ]:
!pip install tensorflow_text

In [2]:
import numpy as np
import pandas as pd

# machine learning libraries
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
print(tf. __version__)

2.15.0



In [3]:
print(tf.test.gpu_device_name())

Import data set from csv

In [4]:
# import data set
df = pd.read_csv("combined_data.csv")
df.head()

# 1 = spam
# 0 = not spam

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...


Find the percantage of spam and not spam labels in data set

In [5]:
# total spam(1) and not spam(0) in DataFrame
print(df['label'].value_counts(), "\n")

# percentages of spam and not spam
print(round(df['label'].value_counts(normalize=True) * 100))

1    43910
0    39538
Name: label, dtype: int64 

1    53.0
0    47.0
Name: label, dtype: float64


To get a 50/50 split we can either over sample or down sample. Given the size of the dataset down sampling will be safe to use. This will help improve efficiency and reduce the risk of overfitting.

In [6]:
# create DataFrames for spam and non-spam to be used for their shape values
df_spam = df[df['label']==1]
df_notSpam = df[df['label']==0]

# print shapes
print("Spam:    ", df_spam.shape[0])
print("Not Spam:", df_notSpam.shape[0])

Spam:     43910
Not Spam: 39538


In [26]:
# create down sampled DataFrame of spam values equal to amount of non-spam values
# by using a sample size equal to the first index in the notSpam shape-tuple

#df_spam_ds = df_spam.sample(df_notSpam.shape[0])

df_spam_ds = df_spam.sample(1500)
df_notSpam_ds = df_notSpam.sample(1500) # new


# print shape
print("Down sampled spam:", df_spam_ds.shape[0])

Down sampled spam: 1500


In [27]:
# create DataFrame with 50/50 split of spam and not spam values by combining original
# notSpam DataFrama with downsampled spam DataFrame

#df_balanced = pd.concat([df_notSpam, df_spam_ds])

df_balanced = pd.concat([df_notSpam_ds, df_spam_ds]) # new

# display new DataFrame counts
df_balanced['label'].value_counts()

0    1500
1    1500
Name: label, dtype: int64

In [24]:
df_balanced.sample(5)

,label,text
24595,1,get it before the rush lookup cdyv current esc...
43781,0,package ftp debian org severity serious twolam...
46424,1,"sick of pil . l . s , we ' ll here the alterna..."
42334,1,while we may have high expectations of our ass...
23692,1,escapelong bescapenumberi bescapenumber cescap...


In [10]:
# create train and test sets
# use stratify to ensure balanced class distrubtion across train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_balanced['text'], df_balanced['label'], stratify=df_balanced['label'])

Use BERT transformer to take string as input and create dictionary of vectors for neural network


In [11]:
# hello BERT
bert_preprocesser = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [16]:
# bert layers for text processing
email_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='email')
preprocessed_email = bert_preprocesser(email_input)
outputs = bert_encoder(preprocessed_email)

# nerual network layers using vectors created by bert transformer
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output']) # dropout to prevent overfitting
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l) # 1 neuron for binary classification

# model
model = tf.keras.Model(inputs=[email_input], outputs = [l])

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 email (InputLayer)          [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['email[0][0]']               
                             128),                                                                
                              'input_word_ids': (None,                                            
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             }                                                                

In [18]:
# configure model for training
model.compile(optimizer='adam',
              loss='binary_crossentropy', # training
              metrics=['accuracy']
              )

In [20]:
# train model
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
33/33 [==============================] - 298s 9s/step - loss: 0.5241 - accuracy: 0.7657
Epoch 2/10
33/33 [==============================] - 296s 9s/step - loss: 0.5105 - accuracy: 0.7705
Epoch 3/10
33/33 [==============================] - 300s 9s/step - loss: 0.5114 - accuracy: 0.7829
Epoch 4/10
33/33 [==============================] - 297s 9s/step - loss: 0.5021 - accuracy: 0.7657
Epoch 5/10
33/33 [==============================] - 295s 9s/step - loss: 0.5017 - accuracy: 0.7714
Epoch 6/10
33/33 [==============================] - 293s 9s/step - loss: 0.5009 - accuracy: 0.7876
Epoch 7/10
33/33 [==============================] - 297s 9s/step - loss: 0.4711 - accuracy: 0.8000
Epoch 8/10
33/33 [==============================] - 299s 9s/step - loss: 0.4745 - accuracy: 0.7943
Epoch 9/10
33/33 [==============================] - 296s 9s/step - loss: 0.4751 - accuracy: 0.7981
Epoch 10/10
33/33 [==============================] - 293s 9s/step - loss: 0.4683 - accuracy: 0.8000
